In [3]:
import numpy as np
from utils import *
import random
import pprint
import copy

In [4]:
!pwd

/Users/niloufar/Desktop/DeepLearning/deep_learning/DL/DeepLearning_coursera/Course5/week1/Dinosaur Island


In [11]:
data = open('dinos.txt', 'r').read()

In [10]:
data

['Aachenosaurus\n',
 'Aardonyx\n',
 'Abdallahsaurus\n',
 'Abelisaurus\n',
 'Abrictosaurus\n',
 'Abrosaurus\n',
 'Abydosaurus\n',
 'Acanthopholis\n',
 'Achelousaurus\n',
 'Acheroraptor\n',
 'Achillesaurus\n',
 'Achillobator\n',
 'Acristavus\n',
 'Acrocanthosaurus\n',
 'Acrotholus\n',
 'Actiosaurus\n',
 'Adamantisaurus\n',
 'Adasaurus\n',
 'Adelolophus\n',
 'Adeopapposaurus\n',
 'Aegyptosaurus\n',
 'Aeolosaurus\n',
 'Aepisaurus\n',
 'Aepyornithomimus\n',
 'Aerosteon\n',
 'AetonyxAfromimus\n',
 'Afrovenator\n',
 'Agathaumas\n',
 'Aggiosaurus\n',
 'Agilisaurus\n',
 'Agnosphitys\n',
 'Agrosaurus\n',
 'Agujaceratops\n',
 'Agustinia\n',
 'Ahshislepelta\n',
 'Airakoraptor\n',
 'Ajancingenia\n',
 'Ajkaceratops\n',
 'Alamosaurus\n',
 'Alaskacephale\n',
 'Albalophosaurus\n',
 'Albertaceratops\n',
 'Albertadromeus\n',
 'Albertavenator\n',
 'Albertonykus\n',
 'Albertosaurus\n',
 'Albinykus\n',
 'Albisaurus\n',
 'Alcovasaurus\n',
 'Alectrosaurus\n',
 'Aletopelta\n',
 'Algoasaurus\n',
 'Alioramus\n',